---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 2 - Network Connectivity

In this assignment you will go through the process of importing and analyzing an internal email communication network between employees of a mid-sized manufacturing company. 
Each node represents an employee and each directed edge between two nodes represents an individual email. The left node represents the sender and the right node represents the recipient.

In [1]:
import networkx as nx

# This line must be commented out when submitting to the autograder
#!head email_network.txt

### Question 1

Using networkx, load up the directed multigraph from `email_network.txt`. Make sure the node names are strings.

*This function should return a directed multigraph networkx graph.*

In [2]:
def answer_one():
    import pandas as pd
    import numpy as np
    df = pd.read_csv("email_network.txt",delimiter='\t')
    df.rename(inplace = True, index=str, columns={'#Sender':'Sender'})
    df['time'] = df['time'].astype(int)
    # The line below contains a bug.
    #G = nx.from_pandas_dataframe(df=df, source='Sender',target='Recipient', edge_attr=True, create_using=nx.MultiDiGraph())
    # Used this way to deal with a bug in the networkx
    G = nx.MultiDiGraph()
    G.add_weighted_edges_from([tuple(d) for d in df[['Sender','Recipient','time']].values])
    return G

In [3]:
answer_one()

### Question 2

How many employees and emails are represented in the graph from Question 1?

*This function should return a tuple (#employees, #emails).*

In [4]:
def answer_two():
    import pandas as pd
    import numpy as np
    G = answer_one()
    n_employees = len(G.nodes(data=all))
    n_emails = len(G.edges(data=all))
    return (n_employees,n_emails)

In [5]:
answer_two()

(167, 82927)

### Question 3

* Part 1. Assume that information in this company can only be exchanged through email.

    When an employee sends an email to another employee, a communication channel has been created, allowing the sender to provide information to the receiver, but not vice versa. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


* Part 2. Now assume that a communication channel established by an email allows information to be exchanged both ways. 

    Based on the emails sent in the data, is it possible for information to go from every employee to every other employee?


*This function should return a tuple of bools (part1, part2).*

In [6]:
def answer_three():
    G = answer_one()
    n = answer_two()
    n_employees = n[0]
    cycle_node_lst = nx.find_cycle(G, orientation='original')
    ans1 = False
    for cycle_nodes in cycle_node_lst:
        if (len(cycle_nodes) == n_employees):
            ans1 = True
    # Part 2 is trivially true
    return (ans1,True)

In [7]:
answer_three()

(False, True)

### Question 4

How many nodes are in the largest (in terms of nodes) weakly connected component?

*This function should return an int.*

In [8]:
def answer_four():
    G = answer_one()
    return len(list(list(nx.weakly_connected_components(G))[0]))

In [9]:
answer_four()

167

### Question 5

How many nodes are in the largest (in terms of nodes) strongly connected component?

*This function should return an int*

In [10]:
def answer_five():
    G = answer_one()
    lst_scp = list(nx.strongly_connected_components(G))
    max_len = 0
    for scp in lst_scp:
        tmp = len(list(scp))
        if (max_len < tmp):
            max_len = tmp
    return max_len

In [11]:
answer_five()

126

### Question 6

Using the NetworkX function strongly_connected_component_subgraphs, find the subgraph of nodes in a largest strongly connected component. 
Call this graph G_sc.

*This function should return a networkx MultiDiGraph named G_sc.*

In [12]:
def answer_six():
    G = answer_one()
    max_c = list(nx.strongly_connected_component_subgraphs(G))[0]
    max_c = (max_c,len(max_c))
    for c in list(nx.strongly_connected_component_subgraphs(G)):
        current = c.nodes(data=all)
        len_current = len(current)
        if (len_current > max_c[1]):
            max_c = (c,len(current))
    return max_c[0]

In [13]:
answer_six()

### Question 7

What is the average distance between nodes in G_sc?

*This function should return a float.*

In [14]:
def answer_seven():
    G = answer_six()
    return nx.average_shortest_path_length(G)

In [15]:
answer_seven()

1.6461587301587302

### Question 8

What is the largest possible distance between two employees in G_sc?

*This function should return an int.*

In [16]:
def answer_eight():
    G = answer_six()
    return nx.diameter(G)

In [17]:
answer_eight()

3

### Question 9

What is the set of nodes in G_sc with eccentricity equal to the diameter?

*This function should return a set of the node(s).*

In [18]:
def answer_nine():
    diameter = answer_eight()
    G = answer_six()
    eccentricity = nx.eccentricity(G)
    rt_lst = []
    for key,val in eccentricity.items():
        if (val == diameter):
            rt_lst.append(str(key))
    return set(rt_lst)

In [19]:
answer_nine()

{'129', '134', '97'}

### Question 10

What is the set of node(s) in G_sc with eccentricity equal to the radius?

*This function should return a set of the node(s).*

In [20]:
def answer_ten():
    G = answer_six()
    radius = nx.radius(G)
    eccentricity = nx.eccentricity(G)
    rt_lst = []
    for key,val in eccentricity.items():
        if (val == radius):
            rt_lst.append(str(key))
    return set(rt_lst)

In [21]:
answer_ten()

{'38'}

### Question 11

Which node in G_sc is connected to the most other nodes by a shortest path of length equal to the diameter of G_sc?

How many nodes are connected to this node?


*This function should return a tuple (name of node, number of satisfied connected nodes).*

In [22]:
def answer_eleven():
    diameter = answer_eight()
    G = answer_six()
    nodes_lst = G.nodes(data=all)
    rt_lst = []
    for node in nodes_lst:
        node = node[0]
        node_dict = nx.shortest_path_length(G,source=node)
        sum_path_eq_dia = 0
        for key, val in node_dict.items():
            if (val == diameter):
                sum_path_eq_dia += 1
        rt_lst.append((node,sum_path_eq_dia))
    rt = sorted(rt_lst, key=lambda x: x[1])[-1]
    # coerce name of node to change from int to str to pass the autograder
    return (str(rt[0]),rt[1])

In [23]:
answer_eleven()

('97', 63)

### Question 12

Suppose you want to prevent communication from flowing to the node that you found in the previous question from any node in the center of G_sc, what is the smallest number of nodes you would need to remove from the graph (you're not allowed to remove the node from the previous question or the center nodes)? 

*This function should return an integer.*

In [24]:
def answer_twelve():
    from networkx.algorithms.connectivity import minimum_st_node_cut
    G = answer_six()
    node = int(answer_eleven()[0])
    center_nodes = nx.center(G)
    cut_nodes = []
    for c_node in center_nodes:
        cut_nodes.append(minimum_st_node_cut(G, s = c_node, t = node))
    return len(list(cut_nodes[0]))

In [25]:
answer_twelve()

5

### Question 13

Construct an undirected graph G_un using G_sc (you can ignore the attributes).

*This function should return a networkx Graph.*

In [26]:
def answer_thirteen():
    G = answer_six()
    G = nx.Graph(G.to_undirected())
    return G

In [27]:
answer_thirteen()

### Question 14

What is the transitivity and average clustering coefficient of graph G_un?

*This function should return a tuple (transitivity, avg clustering).*

In [28]:
def answer_fourteen():  
    G = answer_thirteen()
    return (nx.transitivity(G),nx.average_clustering(G))

In [29]:
answer_fourteen()

(0.570111160700385, 0.6975272437231418)